In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Lakehone/ML_Pipeline.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Lakehone"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "761c242e294f115aac54a7f4acc6b114c878e85a"

In [2]:
# remote modify origin --local access_key_id 761c242e294f115aac54a7f4acc6b114c878e85a
#dvc remote modify origin --local secret_access_key 761c242e294f115aac54a7f4acc6b114c878e85a

In [3]:
import os

In [4]:
%pwd

'c:\\Users\\Sadeeq\\Desktop\\Krish Naik\\End to End Project\\ML_Pipeline\\research'

In [5]:
os.chdir("../")
%pwd

'c:\\Users\\Sadeeq\\Desktop\\Krish Naik\\End to End Project\\ML_Pipeline'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from src.mlproject.constants import *
from src.mlproject.utils import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
            
        create_directories([config.root_dir])
            
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Lakehone/ML_Pipeline.mlflow"

        )
        
        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log params & metrics to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"rmse": rmse, "r2": r2, "mae": mae})

            # Save model manually & log as artifact (works on DagsHub & any MLflow)
            joblib.dump(model, "model.pkl")
            mlflow.log_artifact("model.pkl")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-19 22:11:19,005: INFO: __init__ - YAML file read successfully from config\config.yaml]
[2025-07-19 22:11:19,016: INFO: __init__ - YAML file read successfully from params.yaml]
[2025-07-19 22:11:19,022: INFO: __init__ - YAML file read successfully from schema.yaml]
[2025-07-19 22:11:19,026: INFO: __init__ - Directory created at artifacts]
[2025-07-19 22:11:19,028: INFO: __init__ - Directory created at artifacts/model_evaluation]
[2025-07-19 22:11:22,417: INFO: __init__ - Data saved to JSON file at artifacts\model_evaluation\metrics.json]
🏃 View run lyrical-hen-747 at: https://dagshub.com/Lakehone/ML_Pipeline.mlflow/#/experiments/0/runs/89ed43c86bda4d53baa287587c234bc5
🧪 View experiment at: https://dagshub.com/Lakehone/ML_Pipeline.mlflow/#/experiments/0
